# Conn Settings

In [43]:
import sqlite3
import pandas as pd
import warnings
import numpy as np

warnings.filterwarnings('ignore')

In [44]:
conn = sqlite3.connect("main.db")

# Import To JournalEntry_Rad

In [45]:
raw = pd.read_csv(r"./source/JournalEntry.csv")

In [46]:
raw.head()

,CompanyId,EntryId,BusinessUnitId,AccountNo,Amount,CurrencyId,AccountingDate,DataType,CVSRADDescription,PROJRADDescription,...,BusinessUnit,Account,AccountType,PeriodId,SourceType,SourceRef1,SourceRef2,Calc1,Calc2,Calc3
0,1.0,1921.0,4400.0,50000,"1,432,487.85",USD,09/30/2020,Financial,NaN,NaN,...,HCD - Eligiblity & Enrollment,Salaries,Expense,2012.0,Conversion,C1109_OPBAL,44447.0,NaN,NaN,NaN
1,1.0,1921.0,8900.0,50000,"86,395.08",USD,09/30/2020,Financial,NaN,NaN,...,LPPF - Local Provider Participation Fund,Salaries,Expense,2012.0,Conversion,C1109_OPBAL,44447.0,NaN,NaN,NaN
2,1.0,1921.0,8950.0,50000,"76,451.65",USD,09/30/2020,Financial,NaN,NaN,...,Section 1115 Waiver Costs,Salaries,Expense,2012.0,Conversion,C1109_OPBAL,44447.0,NaN,NaN,NaN
3,1.0,1921.0,9500.0,50000,-5.98,USD,09/30/2020,Financial,NaN,NaN,...,Finance,Salaries,Expense,2012.0,Conversion,C1109_OPBAL,44447.0,NaN,NaN,NaN
4,1.0,1921.0,4200.0,50000,"97,178.14",USD,09/30/2020,Financial,NaN,NaN,...,HCD - Service Delivery Operations,Salaries,Expense,2012.0,Conversion,C1109_OPBAL,44447.0,NaN,NaN,NaN


In [47]:
df = raw.copy()
df = df.reset_index(drop=True)
df['Id'] = df.index + 1
df.head(2)

,CompanyId,EntryId,BusinessUnitId,AccountNo,Amount,CurrencyId,AccountingDate,DataType,CVSRADDescription,PROJRADDescription,...,Account,AccountType,PeriodId,SourceType,SourceRef1,SourceRef2,Calc1,Calc2,Calc3,Id
0,1.0,1921.0,4400.0,50000,"1,432,487.85",USD,09/30/2020,Financial,NaN,NaN,...,Salaries,Expense,2012.0,Conversion,C1109_OPBAL,44447.0,NaN,NaN,NaN,1
1,1.0,1921.0,8900.0,50000,"86,395.08",USD,09/30/2020,Financial,NaN,NaN,...,Salaries,Expense,2012.0,Conversion,C1109_OPBAL,44447.0,NaN,NaN,NaN,2


In [48]:
# Function to determine fiscal year based on date
def get_fiscal_year(date):
    if pd.isna(date):
        return "UnkownFiscalYear"

    year = date.year
    
    # Define fiscal year start and end based on the current year
    fiscal_start = pd.Timestamp(f'{year-1}-10-01')
    fiscal_end = pd.Timestamp(f'{year}-09-30')

    # If date is between October 1st of the previous year and September 30th of the current year
    if fiscal_start <= date <= fiscal_end:
        return f'FY{str(year)[-2:]}'  # Current year as fiscal year
    else:
        return f'FY{str(year + 1)[-2:]}'  # Next year as fiscal year
    

In [49]:
df = raw.copy()
df.columns = df.columns.str.replace(" ", "")
df = df.reset_index(drop=True)
df["Id"] = df.index + 1
df["AccountingDate"] = pd.to_datetime(
    df["AccountingDate"], format="%m/%d/%Y", errors="coerce"
)
df["FiscalYear"] = df["AccountingDate"].apply(get_fiscal_year)
df.head(2)

,CompanyId,EntryId,BusinessUnitId,AccountNo,Amount,CurrencyId,AccountingDate,DataType,CVSRADDescription,PROJRADDescription,...,AccountType,PeriodId,SourceType,SourceRef1,SourceRef2,Calc1,Calc2,Calc3,Id,FiscalYear
0,1.0,1921.0,4400.0,50000,"1,432,487.85",USD,2020-09-30,Financial,NaN,NaN,...,Expense,2012.0,Conversion,C1109_OPBAL,44447.0,NaN,NaN,NaN,1,FY20
1,1.0,1921.0,8900.0,50000,"86,395.08",USD,2020-09-30,Financial,NaN,NaN,...,Expense,2012.0,Conversion,C1109_OPBAL,44447.0,NaN,NaN,NaN,2,FY20


In [50]:
df_split = df.copy()
cols = [col for col in df_split.columns if "RAD" in col and "Description" not in col]

In [51]:

df_split[cols] = df_split[cols].applymap(lambda x: str(x) if not pd.isna(x) else np.nan)
df_split["Combined"] = df_split[cols].apply(
    lambda row: "|".join([str(x) for x in row if not pd.isna(x)]), axis=1
)
df_split = df_split.assign(value=df_split['Combined'].str.split('|')).explode('value')
df_result = df_split[['Id', 'value']].reset_index(drop=True)

In [52]:
df_result.replace("", float("NaN"), inplace=True)
df_result.dropna(inplace=True)
df_result.rename(columns={'value': 'RADId', 'Id': 'JournalEntryId'}, inplace=True)
df_result

,JournalEntryId,RADId
0,1,REG
1,2,REG
2,3,REG
3,4,REG
4,5,PTO
...,...,...
224530,210123,CH RZ
224531,210124,CH RZ
224534,210127,CELL
224544,210137,LONSTA


# Import To JournalEntry

In [53]:
je = df[
    [
        "Id",
        "FiscalYear",
        "CompanyId",
        "EntryId",
        "BusinessUnitId",
        "AccountNo",
        "Amount",
        "AccountingDate",
        "Remarks",
    ]
].copy()
je["EntryId"] = je["EntryId"].fillna(0).astype(int)
je["AccountNo"] = je["AccountNo"].fillna(0).astype(str).replace(r"\.0$", "", regex=True)
je["BusinessUnitId"] = je["BusinessUnitId"].fillna(0).astype(str).replace(r"\.0$", "", regex=True)
je["CompanyId"] = je["CompanyId"].fillna(0).astype(str).replace(r"\.0$", "", regex=True)
je["Amount"] = je["Amount"].str.replace(',', '').fillna(0).astype(float)
je

,Id,FiscalYear,CompanyId,EntryId,BusinessUnitId,AccountNo,Amount,AccountingDate,Remarks
0,1,FY20,1,1921,4400,50000,1432487.85,2020-09-30,NaN
1,2,FY20,1,1921,8900,50000,86395.08,2020-09-30,NaN
2,3,FY20,1,1921,8950,50000,76451.65,2020-09-30,NaN
3,4,FY20,1,1921,9500,50000,-5.98,2020-09-30,NaN
4,5,FY20,1,1921,4200,50000,97178.14,2020-09-30,NaN
...,...,...,...,...,...,...,...,...,...
210133,210134,FY24,1,19374,1000,11100,-547508.44,2024-09-27,AP Check Run 6387 dated 09/27/2024
210134,210135,FY24,1,19374,1000,20000,547508.44,2024-09-27,AP Check Run 6387 dated 09/27/2024
210135,210136,FY24,1,19427,1000,20000,-75000.00,2024-09-23,Batch 14081
210136,210137,FY24,1,19427,4100,65000,75000.00,2024-09-23,EHR Adoption V 1 17078 C2671 C2671_ELECTRONIC ...


In [54]:
def import_func(df: pd.DataFrame(), table: str, conn: sqlite3.Connection) -> None:
    try:
        cursor = conn.cursor()
        cursor.execute(f'DELETE FROM {table}')
        conn.commit()
        df.to_sql(table, conn, if_exists='append', index=False)
    except Exception as error:
        print(error)
    finally:
        cursor.close()
        conn.close()

table = "JournalEntry"
import_func(je, table, conn)
